In [1]:
def fib(n):
    """nを0以上の整数とする
       n番目のフィボナッチ数を返す"""
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [2]:
def fastFib(n, memo = {}):
    """nを0以上の整数とする、memoは再帰呼び出しによってのみ使われる
       n番目のフィボナッチ数を返す"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

In [3]:
%time fastFib(120)

CPU times: user 529 µs, sys: 195 µs, total: 724 µs
Wall time: 617 µs


8670007398507948658051921L

In [4]:
class Item(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = float(v)
        self.weight = float(w)
    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def getWeight(self):
        return self.weight
    def __str__(self):
        result = '<' + self.name + ', ' + str(self.value) + ', ' + str(self.weight) + '>'
        return result

In [5]:
def maxVal(toConsider, avail):
    """toConsiderを品物のリスト、availを重さとする。
       それらをパラメータとする0/1ナップサック問題の解である
       総重量と品物のリストからなるタプルを返す"""
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getWeight() > avail:
        # 右側の分岐のみを探索する
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        # 左側の分岐を探索する
        withVal, withToTake = maxVal(toConsider[1:], avail - nextItem.getWeight())
        withVal += nextItem.getValue()
        # 右側の分岐を探索する
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        # いい方の分岐を選択
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

In [6]:
def smallTest():
    names = ['a', 'b', 'c', 'd']
    vals = [6, 7, 8, 9]
    weights = [3, 3, 2, 5]
    Items = []
    for i in range(len(vals)):
        Items.append(Item(names[i], vals[i], weights[i]))
    val, taken = maxVal(Items, 5)
    for item in taken:
        print item
    print '選択した項目の合計金額 =', val

In [7]:
%time smallTest()

<c, 8.0, 2.0>
<b, 7.0, 3.0>
選択した項目の合計金額 = 15.0
CPU times: user 282 µs, sys: 96 µs, total: 378 µs
Wall time: 312 µs


In [8]:
import random

In [9]:
def buildManyItems(numItems, maxVal, maxWeight):
    items = []
    for i in range(numItems):
        items.append(Item(str(i), random.randint(1, maxVal), random.randint(1, maxWeight)))
    return items

In [10]:
def bigTest(numItems):
    items = buildManyItems(numItems, 10, 10)
    val, taken = maxVal(items, 40)
    print '選択した項目'
    for item in taken:
        print item
    print '選択した項目の合計金額 =', val

In [11]:
%time bigTest(10)

選択した項目
<9, 10.0, 1.0>
<8, 7.0, 8.0>
<7, 10.0, 4.0>
<6, 8.0, 5.0>
<4, 6.0, 7.0>
<2, 4.0, 2.0>
<1, 4.0, 4.0>
<0, 9.0, 7.0>
選択した項目の合計金額 = 58.0
CPU times: user 5.35 ms, sys: 2.56 ms, total: 7.91 ms
Wall time: 5.62 ms


In [12]:
%time bigTest(40)

KeyboardInterrupt: 

In [13]:
def fastMaxVal(toConsider, avail, memo = {}):
    """toConsiderを品物のリスト、availを重さ、
       memoは再帰呼び出しによってのみ使われるとする。
       それらをパラメータとする0/1ナップサック問題の解である
       総重量と品物のリストからなるタプルを返す"""
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getWeight() > avail:
        # 右側の分岐のみを探索する
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        # 左側の分岐を探索する
        withVal, withToTake = fastMaxVal(toConsider[1:], avail - nextItem.getWeight(), memo)
        withVal += nextItem.getValue()
        # 右側の分岐を探索する
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:], avail, memo)
        # いい方の分岐を選択
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    return result

In [14]:
def bigTest(numItems):
    items = buildManyItems(numItems, 10, 10)
    val, taken = fastMaxVal(items, 1000)
    print '選択した項目'
    for item in taken:
        print item
    print '選択した項目の合計金額 =', val

In [15]:
def buildManyItems(numItems, maxVal, maxWeight):
    items = []
    for i in range(numItems):
        items.append(Item(str(i), random.randint(1, maxVal), random.randint(1, maxWeight) * random.random()))
    return items

In [16]:
%time bigTest(10)

選択した項目
<9, 7.0, 1.70641041467>
<8, 4.0, 1.3935006505>
<7, 7.0, 3.92507688218>
<6, 4.0, 0.853119599545>
<5, 10.0, 1.76864903543>
<4, 3.0, 1.47296246822>
<3, 2.0, 0.275182609364>
<2, 5.0, 0.215008281661>
<1, 2.0, 1.75451592515>
<0, 6.0, 4.95551705624>
選択した項目の合計金額 = 50.0
CPU times: user 7.9 ms, sys: 2.11 ms, total: 10 ms
Wall time: 8.19 ms


In [17]:
%time bigTest(40)

KeyboardInterrupt: 